# The files have been generated in Databricks and provided on the S3 bucket

In [70]:
#!pip install boto3 # not required for Jupyter Notebooks on AWS infrastructure

In [71]:
import boto3

boto_session = boto3.Session() # Grabs session details directly from aws configuration in EC2 instance running the Notebook server
boto_session

Session(region_name='eu-central-1')

In [72]:
# Define variables for paths
s3_path = f"s3://sjf-project/"
output_path = s3_path + "estimator"

print(f"Using bucket '{s3_path}'.")

Using bucket 's3://sjf-project/'.


In [73]:
#!pip install sagemaker
import sagemaker

# Create a SageMaker Session
sagemaker_session = sagemaker.Session(boto_session=boto_session)

#The following IAM role ARN was taken from the flutz notebook listed in the SageMaker console:
sm_execution_role = 'arn:aws:iam::898627427171:role/service-role/AmazonSageMaker-ExecutionRole-20201106T104926'

## Select cells for either XGBooster or Linear Learner

In [74]:
# # Use a previously-built, AWS XGBoost model for training

# from sagemaker.amazon.amazon_estimator import get_image_uri
# # container = get_image_uri(***insert correct arguments here***) 

# container = get_image_uri(region_name=boto_session.region_name,
#                           repo_name='xgboost',
#                           repo_version='1.0-1')

In [75]:
# Use the predefined Linear learner model


ecr_path = "664544806723.dkr.ecr.eu-central-1.amazonaws.com"

linear = sagemaker.estimator.Estimator(f"{ecr_path}/linear-learner:latest",
                              sm_execution_role,
                              train_instance_count=1,
                              train_instance_type="ml.m5.xlarge",
                              output_path=output_path,
                              sagemaker_session=sagemaker.Session(),
                              base_job_name="sjf-linear")

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [76]:

# # From https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html 
# # XGBoost ecr path is: 492215442770.dkr.ecr.eu-central-1.amazonaws.com
# ecr_path = '492215442770.dkr.ecr.eu-central-1.amazonaws.com'

# #'ml.m4.xlarge') | 'ml.t2.medium' || train_instance_type= "ml.m5.large" -> "ml.m5.xlarge"
# #Instance Name	vCPU	ECU	RAM   	Instance-GB	Linux/UNIX-Nutzung
# #t2.medium	2	Var.	4 GiB	Nur EBS 	0,0536 USD pro Stunde
# #m4.xlarge	4	13	16 GiB	Nur EBS 	0,24 USD pro Stunde
# #m5.large	2	10	8 GiB	Nur EBS 	0,115 USD pro Stunde
# #m5.xlarge	4	16	16 GiB	Nur EBS 	0,23 USD pro Stunde
# xgboost = sagemaker.estimator.Estimator(f"{ecr_path}/sagemaker-xgboost:1.0-1-cpu-py3",
#                               sm_execution_role,
#                               train_instance_count=1,
#                               train_instance_type="ml.m5.xlarge",
#                               output_path=output_path,
#                               sagemaker_session=sagemaker_session,
#                               base_job_name="sjf-xgboost")


In [77]:
# Create pointers to the S3 train and test datasets

from sagemaker.session import s3_input

Train = 'train_data_trial.csv'
Valid = 'valid_data_trial.csv'

s3_input_train = sagemaker.session.s3_input(s3_data=f"{s3_path}{Train}", content_type="text/csv")
s3_input_valid = sagemaker.session.s3_input(s3_data=f"{s3_path}{Valid}", content_type="text/csv")
#s3_input_train = sagemaker.session.s3_input(s3_data=f"{s3_path}train_sample.csv", content_type="text/csv")
#s3_input_valid = sagemaker.session.s3_input(s3_data=f"{s3_path}valid_sample.csv", content_type="text/csv")
#s3_input_test = sagemaker.session.s3_input(s3_data=f"{s3_path}test.csv", content_type="text/csv")

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


## Select cells for XGBooster or linear learner

In [78]:
# # Create an XGBoost Estimator

# # From https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html 
# # XGBoost ecr path is: 492215442770.dkr.ecr.eu-central-1.amazonaws.com
# ecr_path = '492215442770.dkr.ecr.eu-central-1.amazonaws.com'

# #'ml.m4.xlarge') | 'ml.t2.medium' || train_instance_type= "ml.m5.large" -> "ml.m5.xlarge"
# #Instance Name	vCPU	ECU	RAM   	Instance-GB	Linux/UNIX-Nutzung
# #t2.medium	2	Var.	4 GiB	Nur EBS 	0,0536 USD pro Stunde
# #m4.xlarge	4	13	16 GiB	Nur EBS 	0,24 USD pro Stunde
# #m5.large	2	10	8 GiB	Nur EBS 	0,115 USD pro Stunde
# #m5.xlarge	4	16	16 GiB	Nur EBS 	0,23 USD pro Stunde
# xgboost = sagemaker.estimator.Estimator(f"{ecr_path}/sagemaker-xgboost:1.0-1-cpu-py3",
#                               sm_execution_role,
#                               train_instance_count=1,
#                               train_instance_type="ml.m5.xlarge",
#                               output_path=output_path,
#                               sagemaker_session=sagemaker_session,
#                               base_job_name="sjf-xgboost")


In [79]:
# # Select the your specific hyperparameters (Optional)

# # From https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html

# xgboost.set_hyperparameters(
#     eta=0.3,                         # default 0.3 range 0 - 1   
#     num_round=30,                    # required | valid values: int | The number of rounds to run the training.
#     objective = 'reg:linear'  # Examples: reg:linear, reg:logistic, multi:softmax, reg:squarederror [default] 
# )

In [80]:
# Select the your specific hyperparameters (Optional) - from Live Coding
linear.set_hyperparameters(
    predictor_type="regressor",
    normalize_data=False,
    mini_batch_size=50
)

In [81]:
# Fit the model

#xgboost.fit({"train": s3_input_train})
linear.fit({"train": s3_input_train})


2020-11-19 15:08:51 Starting - Starting the training job...
2020-11-19 15:08:55 Starting - Launching requested ML instances......
2020-11-19 15:10:04 Starting - Preparing the instances for training...
2020-11-19 15:10:38 Downloading - Downloading input data...
2020-11-19 15:11:21 Training - Training image download completed. Training in progress.
2020-11-19 15:11:21 Uploading - Uploading generated training model
2020-11-19 15:11:21 Failed - Training job failed
.Docker entrypoint called with argument(s): train
Running default environment configuration script
[11/19/2020 15:11:16 INFO 139671278114624] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'100

UnexpectedStatusException: Error for Training job sjf-linear-2020-11-19-15-08-51-231: Failed. Reason: ClientError: Found at least 1 entries in file /opt/ml/input/data/train/train_data_trial.csv with missing values:
Row 7393, column 5

In [ ]:
# Deploy your model to an endpoint to perform predictions

Predictor = linear.deploy(
#Predictor = xgboost.deploy(
    initial_instance_count = 1, 
    instance_type = 'ml.t2.medium')
    

In [ ]:
Predictor.endpoint

In [ ]:
# Configure the predictor's serializer and deserializer

from sagemaker.predictor import csv_serializer, csv_deserializer

Predictor.serializer = csv_serializer
Predictor.deserializer = csv_deserializer
Predictor.content_type = "text/csv"

In [ ]:
Predictor.endpoint

# Predictions

In [ ]:
# Import the validate dataset into a dataframe

import pandas as pd

validate_df = pd.read_csv(f"{s3_path}{Valid}", header=None)
display(validate_df)

In [ ]:
train_df = pd.read_csv(f"{s3_path}{Train}", header=None)
display(train_df)

In [ ]:
validate_df.iloc[0, 1:]

In [ ]:
prediction = Predictor.predict(validate_df.iloc[0:, 1:].values)
#len(prediction[0])

In [ ]:
#y_predicted = list(map(lambda pred: round(float(pred),1), prediction[0])) # prediction-Werte werde auf eine Kommastelle gerundet
y_predicted = list(map(float, prediction[0]))
display(y_predicted)

In [ ]:
y_true = validate_df[0].values
display(y_true)

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_true, y_predicted))

In [ ]:
from sklearn.metrics import explained_variance_score
print(explained_variance_score(y_true, y_predicted))

In [ ]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_true, y_predicted))

In [ ]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_true, y_predicted))

In [ ]:
pred=[]
pred_df = pd.DataFrame(y_predicted, columns=['prediction'])
pred_df['ground_truth']=y_true
display(pred_df.tail(10))

In [ ]:
import matplotlib.pyplot as plt

x = y_true
y = y_predicted
plt.scatter(x, y, color="red")
plt.title("IMDb Ratings Prediction Model 1840 rows & 409 columns")
plt.xlabel("y_true")
plt.ylabel("y_prediction")
plt.xlim([1,10])
plt.ylim([1,10])
plt.plot([1,10],[1,10], color="blue")
plt.grid()
plt.show()